<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Otros/Ej_3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy
!pip install numpy scipy matplotlib streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.

In [4]:
%%writefile app.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d
import requests
from io import BytesIO
import yt_dlp as youtube_dl
import subprocess
import os
import shutil
from scipy.fftpack import fft

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 3: Modulación en Amplitud (AM) con Audio",
                          "Punto 4.1: THD y Factor de Potencia",
                          "Punto 4.2: THD y Factor de Potencia"))
if punto == "Punto 3: Modulación en Amplitud (AM) con Audio":
    st.write("### Punto 3: Modulación en Amplitud (AM) con Audio")
# Función para descargar audio de YouTube en formato MP3
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    return filename

# Función para convertir MP3 a WAV usando ffmpeg
def convert_mp3_to_wav(mp3_filename, wav_filename):
    subprocess.call(['ffmpeg', '-y', '-i', mp3_filename, wav_filename])

# Interfaz de Streamlit
st.title("Modulación de Audio desde YouTube y Escuchar Segmento")

# Entrada para el enlace de YouTube
video_url = st.text_input("Ingresa el enlace de una canción en YouTube:")

# Parámetros ajustables con sliders
start_time = st.slider("Tiempo de inicio del segmento (segundos)", 0, 60, 35)     # Tiempo de inicio del segmento

# Procesamiento del archivo de audio si se ha ingresado un enlace
if video_url:
      try:
          name = "audio_downloaded"
          mp3_filename = download_ytvid_as_mp3(video_url, name)
          wav_filename = f"{name}.wav"
          convert_mp3_to_wav(mp3_filename, wav_filename)

          # Leer el archivo WAV
          x, fm = sf.read(wav_filename)
          x_mono = np.mean(x, axis=1) if x.ndim > 1 else x
          ns = len(x_mono) / fm

          # Segmento seleccionado por el usuario
          end_time = start_time + 5
          start_index = int(start_time * fm)
          end_index = int(end_time * fm)

          x_mono_segment = x_mono[start_index:end_index]
          t_segment = np.arange(start_time, end_time, 1 / fm)

          # Reproducir el segmento seleccionado
          st.write(f"Reproduciendo segmento de {start_time} a {end_time} segundos.")

          # Guardar el segmento seleccionado como archivo temporal WAV
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_wav:
              sf.write(temp_wav.name, x_mono_segment, fm)
              st.audio(temp_wav.name, format='audio/wav')

          # Generar la señal portadora
          Ac = max(abs(x_mono_segment)) / 0.8  # Amplitud de la portadora
          c_segment = Ac * np.sin(2 * np.pi * 10000 * t_segment)  # Señal portadora

          # Señal modulada AM
          y = (1 + x_mono_segment / Ac) * c_segment

          # Graficar señal portadora y mensaje
          st.write("### Señal de Portadora y Mensaje")
          fig, ax = plt.subplots()
          ax.plot(t_segment, c_segment, label='Portadora', alpha=0.7)
          ax.plot(t_segment, x_mono_segment, label='Mensaje', alpha=0.7)
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Graficar la señal modulada
          st.write("### Señal Modulada en AM")
          fig, ax = plt.subplots()
          ax.plot(t_segment, y, label='Modulada')
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          st.pyplot(fig)

          # Espectros de las señales
          st.write("### Espectros de las Señales")
          Xfc = np.fft.rfft(c_segment)
          Xfm = np.fft.rfft(x_mono_segment)
          Xfy = np.fft.rfft(y)
          vfre = np.fft.rfftfreq(len(c_segment), 1 / fm)

          fig, ax = plt.subplots()
          ax.plot(vfre, np.abs(Xfy), label='Modulada', alpha=0.7)
          ax.plot(vfre, np.abs(Xfm), label='Mensaje', alpha=0.7)
          ax.plot(vfre, np.abs(Xfc), label='Portadora', alpha=0.7)
          ax.set_xlabel('Frecuencia [Hz]')
          ax.set_ylabel('|X(f)|')
          ax.set_ylim([min(np.abs(Xfm)), 1.1 * max(np.abs(Xfm))])
          ax.legend()
          ax.grid(True)
          st.pyplot(fig)

      except Exception as e:
          st.error(f"Error al procesar el audio: {e}")
elif punto == "Punto 4.1: THD y Factor de Potencia":
    st.write("### Punto 4.1: THD y Factor de Potencia")
    st.markdown("""
**1. Consulte en que consiste la distorcion total de armonicos(Total Harmonic Distortion-(THD)) y el factor de potencia en un circuito electrico.**

**THD:**
Consiste en que se parte de un sistema ideal en el que la función de transferencia es lineal e invariante en el tiempo.
Cuando una señal sinusoidal de frecuencia ω pasa por un dispositivo no ideal y no lineal, se añade contenido adicional en múltiplos nω (armónicos) de la frecuencia original.
La THD es una medida de ese contenido de señal adicional no presente en la señal de entrada. Esta puede ser THD indicándonos con respecto al voltaje o también puede ser THDi, la cual es con respecto a la corriente.

**FP:**
Es la relación entre la potencia real y la potencia aparente en un circuito o sistema de distribución. La potencia real se mide en W o kW, y es la que utilizan los motores, las luces y otros dispositivos para producir trabajo útil. La potencia aparente se mide en VA o kVA, que es la suma vectorial de la potencia real (trabajo útil), la potencia reactiva (almacenada y liberada por inductores y condensadores y muestra el desplazamiento de fase entre las formas de onda de corriente y tensión), y la potencia armónica (la cual se pierde por distorsión armónica).
""")
    st.markdown("""
**2. ¿Como puede calcularse el THD desde la FFT?**

**Los pasos a seguir serían:**

1) **Realizar la FFT:** Esto transforma la señal del dominio del tiempo al dominio de la frecuencia, permitiendo identificar las diferentes componentes de frecuencia, incluyendo la fundamental y los armónicos.

2) **Identificar la frecuencia fundamental:** Una vez que se tiene el espectro de frecuencias, se debe identificar la frecuencia fundamental $V_1$, que es la frecuencia más baja y representa la señal principal.

3) **Calcular las amplitudes de los armónicos:** A partir del espectro de la FFT, se deben extraer las amplitudes de los armónicos $V_n$, donde $n$ es el número del armónico, comenzando desde el armónico 2 y tomando los componentes de frecuencia que son múltiplos enteros de la frecuencia fundamental.

4) **Aplicar la fórmula del THD:**

$$
THD = \\frac{\\sqrt{V_2^2 + V_3^2 + \\cdots + V_n^2}}{V_1}
$$

donde $V_n$ es el valor del $n$-ésimo armónico de la señal de salida del circuito (en términos de tensión), y $V_1$ es la componente fundamental.
""")
    st.markdown("""
**3. ¿Como puede calcularse la distorsión del factor de potencia con base al THD?**

El factor de potencia de distorsión es la relación entre la potencia real y la potencia aparente debido a la distorsión armónica total (THD). No se pueden añadir condensadores a un circuito para compensar el factor de potencia de distorsión. La impedancia de los condensadores disminuye con la frecuencia. Por lo tanto, un condensador puede convertirse en un sumidero de armónicos de alta frecuencia. Se utilizan tipos especiales de transformadores o filtros armónicos sintonizados que constan de condensadores e inductores para corregir el factor de potencia de distorsión.

El factor de potencia de distorsión se calcula de la siguiente manera:

$$
PF_{THD} = \\sqrt{\\frac{1}{1 + (THD)^2}}
$$
""")
elif punto == "Punto 4.2: THD y Factor de Potencia":
    st.write("### Punto 4.2: THD y Factor de Potencia")
    # Selección interactiva de los valores de R y C
    R = st.slider("Selecciona el valor de la resistencia (R) en ohmios", 0, 100, 50, step=10)
    C = st.slider("Selecciona el valor de la capacitancia (C) en microfaradios (μF)", 0.0, 1000.0, 100.0, step=100.0)
    C = C * 1e-6

    # Simulación de THD y factor de potencia
    st.write("Simulando THD y factor de potencia para un rectificador de onda completa...")
    # Parámetros de simulación
    Fs = 10000  # Frecuencia de muestreo
    T = 1 / 50  # Periodo de la señal (50 Hz)
    t = np.linspace(0, 2 * T, int(Fs * 2 * T))  # Tiempo de simulación corregido
    f = 50  # Frecuencia fundamental de la señal de entrada

    # Generar la señal de voltaje de entrada (onda seno)
    Vin = np.sin(2 * np.pi * f * t)

    # Simulación de rectificador de onda completa con carga resistiva
    I_R = np.abs(Vin / R)  # Corriente a través de la carga resistiva

    # Simulación de rectificador de onda completa con carga RC
    V_C = np.zeros_like(t)  # Inicializar voltaje en el condensador
    I_RC = np.zeros_like(t)  # Inicializar corriente en la carga RC

    # Simulación del voltaje y la corriente en la carga RC
    for i in range(1, len(t)):
        V_C[i] = V_C[i-1] + (Vin[i-1] - V_C[i-1]) / (R * C) * (t[i] - t[i-1])
        I_RC[i] = np.abs((Vin[i] - V_C[i]) / R)

    # Calcular la FFT de la corriente
    def calculate_fft(signal, Fs):
        N = len(signal)
        fft_values = fft(signal)
        fft_magnitude = 2.0 / N * np.abs(fft_values[:N // 2])
        freqs = np.fft.fftfreq(N, 1 / Fs)[:N // 2]
        return freqs, fft_magnitude

    # Calcular la THD a partir de la FFT
    def calculate_thd(signal, Fs):
        freqs, magnitudes = calculate_fft(signal, Fs)
        fundamental_index = np.argmax(magnitudes)  # Detectar la frecuencia fundamental
        fundamental = magnitudes[fundamental_index]  # Componente de la frecuencia fundamental
        harmonic_sum = np.sqrt(np.sum(magnitudes[fundamental_index+1:]**2))  # Suma de los armónicos
        thd = harmonic_sum / fundamental
        return thd

    # Calcular THD y factor de potencia para ambas configuraciones
    thd_resistive = calculate_thd(I_R, Fs)
    thd_rc = calculate_thd(I_RC, Fs)

    # Factor de potencia con THD (sólo por distorsión, no por desfase)
    def power_factor(thd):
        return 1 / np.sqrt(1 + thd**2)

    pf_resistive = power_factor(thd_resistive)
    pf_rc = power_factor(thd_rc)

    # Graficar las corrientes
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

    ax1.plot(t, I_R, label='Carga Resistiva (R)')
    ax1.set_title('Corriente para Carga Resistiva (R)')
    ax1.set_xlabel('Tiempo [s]')
    ax1.set_ylabel('Corriente [A]')
    ax1.grid()

    ax2.plot(t, I_RC, label='Carga RC en Serie', color='orange')
    ax2.set_title('Corriente para Carga RC en Serie')
    ax2.set_xlabel('Tiempo [s]')
    ax2.set_ylabel('Corriente [A]')
    ax2.grid()

    fig.tight_layout()
    st.pyplot(fig)

    # FFT para carga resistiva
    freqs_R, fft_magnitude_R = calculate_fft(I_R, Fs)

    # FFT para carga RC
    freqs_RC, fft_magnitude_RC = calculate_fft(I_RC, Fs)

    # Graficar FFT para ambas cargas
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

    # Gráfico de FFT para la carga resistiva
    ax1.plot(freqs_R, fft_magnitude_R)
    ax1.set_title('Espectro de Fourier - Carga Resistiva (R)')
    ax1.set_xlabel('Frecuencia [Hz]')
    ax1.set_ylabel('Magnitud')
    ax1.grid()

    # Gráfico de FFT para la carga RC
    ax2.plot(freqs_RC, fft_magnitude_RC, color='orange')
    ax2.set_title('Espectro de Fourier - Carga RC en Serie')
    ax2.set_xlabel('Frecuencia [Hz]')
    ax2.set_ylabel('Magnitud')
    ax2.grid()

    fig.tight_layout()
    st.pyplot(fig)

    # Mostrar los resultados de THD y factor de potencia
    st.write(f"**THD para carga resistiva**: {thd_resistive * 100:.2f}%")
    st.write(f"**Factor de potencia para carga resistiva**: {pf_resistive:.2f}")
    st.write(f"**THD para carga RC**: {thd_rc * 100:.2f}%")
    st.write(f"**Factor de potencia para carga RC**: {pf_rc:.2f}")

Writing app.py


In [5]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://2480-35-194-251-214.ngrok-free.app" -> "http://localhost:5011"
